In [1]:
pip install "snowflake-connector-python[pandas]"

Note: you may need to restart the kernel to use updated packages.


In [56]:
#import libraries needed
from snowflake.snowpark.session import Session
import os

#check to ensure you are in your virtual Python 3.8 env
print (os.environ['CONDA_DEFAULT_ENV'])

#config to connect to Snowflake Instance
snowflake_param_dict = {
    #'account':,   #update with your Snowflake info
    #'user':,       #update with your Snowflake info         
    #'password':,    #update with your Snowflake info   
    #'role': 'SYSADMIN', 
    #'warehouse':,  #update with your Snowflake info
    #'database':,    #update with your Snowflake info
    #'schema': }

session = Session.builder.configs(snowflake_param_dict).create()

py38_env


In [57]:
# RAW LAYERS
sf_df_customers = session.table('tmg2546_DW_CAPSTONE.ELT_STAGE.CUSTOMERS_DW')
sf_df_orders = session.table('tmg2546_DW_CAPSTONE.ELT_STAGE.ORDERS_DW')
sf_df_order_items = session.table('tmg2546_DW_CAPSTONE.ELT_STAGE.ORDER_ITEMS_DW')
sf_df_payments = session.table('tmg2546_DW_CAPSTONE.ELT_STAGE.PAYMENTS_DW')
sf_df_products = session.table('tmg2546_DW_CAPSTONE.ELT_STAGE.PRODUCTS_DW')

In [58]:
# use this to make sure the data is pulled from the raw Snowflake layer and is correctly shown
sf_df_customers.show(5)

--------------------------------------------------------------------------------------
|"CUSTOMER_ID"  |"CUSTOMER_ZIP_CODE_PREFIX"  |"CUSTOMER_CITY"     |"CUSTOMER_STATE"  |
--------------------------------------------------------------------------------------
|hCT0x9JiGXBQ   |58125                       |varzea paulista     |SP                |
|PxA7fv9spyhx   |3112                        |armacao dos buzios  |RJ                |
|g3nXeJkGI0Qw   |4119                        |jandira             |SP                |
|EOEsCQ6QlpIg   |18212                       |uberlandia          |MG                |
|mVz5LO2Vd6cL   |88868                       |ilhabela            |SP                |
--------------------------------------------------------------------------------------



In [59]:
# SILVER LAYERS
sf_df_customer_transactions = session.table('tmg2546_DW_CAPSTONE.EDW_SILVER_LAYER.CUSTOMER_TRANSACTIONS')
sf_df_orders_info_table = session.table('tmg2546_DW_CAPSTONE.EDW_SILVER_LAYER.ORDERS_INFO_TABLE')

In [60]:
# use this to make sure the data is pulled from the silver Snowflake layer and is correctly shown
sf_df_customer_transactions.show(5)

---------------------------------------------------------------------------------------------------------------------
|"CUSTOMER_ID"  |"ADDRESS"               |"ORDER_ID"    |"PAYMENT_TYPE"  |"PAYMENT_INSTALLMENTS"  |"PAYMENT_VALUE"  |
---------------------------------------------------------------------------------------------------------------------
|hCT0x9JiGXBQ   |varzea paulista, SP     |Axfy13Hk4PIk  |credit_card     |1                       |259.14           |
|PxA7fv9spyhx   |armacao dos buzios, RJ  |v6px92oS8cLG  |credit_card     |8                       |382.39           |
|g3nXeJkGI0Qw   |jandira, SP             |Ulpf9skrhjfm  |credit_card     |4                       |249.25           |
|EOEsCQ6QlpIg   |uberlandia, MG          |bwJVWupf2keN  |credit_card     |2                       |27.79            |
|mVz5LO2Vd6cL   |ilhabela, SP            |Dd0QnrMk9Cj5  |credit_card     |1                       |76.15            |
--------------------------------------------------------

In [61]:
# use this to make sure the data is pulled from the silver Snowflake layer and is correctly shown
sf_df_orders_info_table.show(5)

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"    |"ORDER_STATUS"  |"ORDER_PURCHASE_TIMESTAMP"  |"ORDER_APPROVED_AT"  |"ORDER_ESTIMATED_DELIVERY_DATE"  |"ORDER_DELIVERED_TIMESTAMP"  |"DAYS_TO_CUSTOMER"  |"SELLER_ID"   |"PRICE"  |"SHIPPING_CHARGES"  |"PRODUCT_ID"  |"PRODUCT_CATEGORY_NAME"  |"ITEM_WEIGHT"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|Axfy13Hk4PIk  |delivered       |2017-10-22 18:57:54         |2017-10-22 19:14:13  |2017-11-09                       |2017-10-26 22:19:52          |4                   |ZWM05J9LcBSF

In [66]:
import pandas as pd

# RAW LAYERS
p_df_customers = sf_df_customers.to_pandas()
p_df_orders = sf_df_orders.to_pandas()
p_df_order_items = sf_df_order_items.to_pandas()
p_df_payments = sf_df_payments.to_pandas()
p_df_products = sf_df_products.to_pandas()

# SILVER LAYERS
p_df_customer_transactions = sf_df_customer_transactions.to_pandas()
p_df_orders_info_table = sf_df_orders_info_table.to_pandas()

In [67]:
# use to make sure the pandas df works correctly for raw layers
p_df_customers.head()

,CUSTOMER_ID,CUSTOMER_ZIP_CODE_PREFIX,CUSTOMER_CITY,CUSTOMER_STATE
0,hCT0x9JiGXBQ,58125,varzea paulista,SP
1,PxA7fv9spyhx,3112,armacao dos buzios,RJ
2,g3nXeJkGI0Qw,4119,jandira,SP
3,EOEsCQ6QlpIg,18212,uberlandia,MG
4,mVz5LO2Vd6cL,88868,ilhabela,SP


In [68]:
# use to make sure the pandas df works correctly for silver layers
p_df_customer_transactions.head()

,CUSTOMER_ID,ADDRESS,ORDER_ID,PAYMENT_TYPE,PAYMENT_INSTALLMENTS,PAYMENT_VALUE
0,hCT0x9JiGXBQ,"varzea paulista, SP",Axfy13Hk4PIk,credit_card,1,259.14
1,PxA7fv9spyhx,"armacao dos buzios, RJ",v6px92oS8cLG,credit_card,8,382.39
2,g3nXeJkGI0Qw,"jandira, SP",Ulpf9skrhjfm,credit_card,4,249.25
3,EOEsCQ6QlpIg,"uberlandia, MG",bwJVWupf2keN,credit_card,2,27.79
4,mVz5LO2Vd6cL,"ilhabela, SP",Dd0QnrMk9Cj5,credit_card,1,76.15


# TWO ANALYTICS MODELS # 
- On raw and silver layers

In [69]:
long_term_cc_accounts = p_df_customer_transactions[(p_df_customer_transactions['PAYMENT_TYPE'] == 'credit_card') & (p_df_customer_transactions['PAYMENT_INSTALLMENTS'] > 10)]

# shows the top ten customers who used a credit card as payment and have many payment installments #

sorted_cc_users = long_term_cc_accounts.sort_values(by='PAYMENT_INSTALLMENTS', ascending=False)
sorted_cc_users.set_index('CUSTOMER_ID')[:10]


###################################### Business Implication ############################################################
# this type of customer most likely will purchase things they cannot immediately afford and could be more willing to make 
# purchases in the future if given a discount / pushed with targeted advertising
########################################################################################################################

,ADDRESS,ORDER_ID,PAYMENT_TYPE,PAYMENT_INSTALLMENTS,PAYMENT_VALUE
CUSTOMER_ID,,,,,
mW4ozLhG9kaU,"brasilia, DF",wbkzxBkuKIqg,credit_card,24,368.96
uYf1PglbQ8Lu,"jaguariuna, SP",6UJUbLRgxFyl,credit_card,24,526.36
mINWUe6dUCqb,"sao joao de meriti, RJ",7qP9whnQos4V,credit_card,24,762.23
gxvhsUR0Q6GV,"jaguariuna, SP",7ioyG9JJMUY0,credit_card,24,526.36
UEUrtxE3R15M,"jaguariuna, SP",8eFU0la0a05k,credit_card,24,526.36
d474CUD0FyA2,"porto alegre, RS",yYoSyqDj9qKu,credit_card,24,84.13
o2kjhhEPemKJ,"vespasiano, MG",TE7IuKQQdVHI,credit_card,24,41.76
lYTK60tSg9LP,"vilhena, RO",kJS8H36LTDZm,credit_card,24,47.47
2WlfWrs3hNVq,"jaguariuna, SP",9thqvoPibrOw,credit_card,24,526.36


In [70]:
relevant_orders_info = p_df_orders_info_table[['ORDER_ID', 'SELLER_ID', 'SHIPPING_CHARGES', 'PRODUCT_ID', 'PRODUCT_CATEGORY_NAME', 'ITEM_WEIGHT']]
relevant_orders_info[:10]

,ORDER_ID,SELLER_ID,SHIPPING_CHARGES,PRODUCT_ID,PRODUCT_CATEGORY_NAME,ITEM_WEIGHT
0,Axfy13Hk4PIk,ZWM05J9LcBSF,84.65,90K0C1fIyQUf,toys,491.0
1,v6px92oS8cLG,IjlpYfhUbRQs,23.79,qejhpMGGVcsl,watches_gifts,440.0
2,bwJVWupf2keN,jWzS0ayv9TGf,30.72,639iGvMyv0De,toys,1450.0
3,Dd0QnrMk9Cj5,l1pYW6GBnPMr,30.66,1lycYGcsic2F,toys,300.0
4,VjTVGzqe8U6R,XePbD5Ocm6Yk,40.26,TpFZIk33uyGn,toys,2800.0
5,xNba906m4TM1,ftbUjJ6Gsazg,98.76,Y0T4Ca4BXK8U,bed_bath_table,3975.0
6,VEDSMTavbQwk,RKad98cTxhSb,66.90,IUykT5z5LcPW,toys,250.0
7,DzNM8wrcMGFH,K0qPVGdA91KO,34.86,0UHKKyYmxWfq,toys,1200.0
8,NpJz7zF9U1ae,2a07LobkzHUx,60.35,MJ82dNJNiV7U,toys,1500.0
9,mHyhzcUeLGj2,coi9xkdo66JI,48.71,wdV7yJzaz3Su,toys,4338.0


## Business Question - Which products cost the most for the company to ship? ##

In [71]:
# Total costs - these cost the company incurs in shipping - influenced by total sales quantity

highest_shipping_charges = pd.DataFrame(relevant_orders_info.groupby(['PRODUCT_CATEGORY_NAME'])['SHIPPING_CHARGES'].sum()).sort_values('SHIPPING_CHARGES', ascending=False)
highest_shipping_charges[:10]

,SHIPPING_CHARGES
PRODUCT_CATEGORY_NAME,
toys,2986483.81
health_beauty,100774.25
bed_bath_table,95180.79
furniture_decor,78996.70
sports_leisure,78250.83
computers_accessories,73718.37
housewares,59011.91
watches_gifts,52077.12
auto,35834.10


In [72]:
## Average shipping costs - may be more representative of expense for shipping since it is not influenced by volume of sales

highest_shipping_averages = pd.DataFrame(relevant_orders_info.groupby(['PRODUCT_CATEGORY_NAME'])['SHIPPING_CHARGES'].mean()).sort_values('SHIPPING_CHARGES', ascending=False)
highest_shipping_averages[:10]

,SHIPPING_CHARGES
PRODUCT_CATEGORY_NAME,
diapers_and_hygiene,108.040000
arts_and_craftmanship,102.737500
food,67.489661
fashion_underwear_beach,63.319375
fashion_shoes,58.568525
industry_commerce_and_business,55.598125
food_drink,52.596308
home_comfort_2,51.960000
small_appliances_home_oven_and_coffee,51.533333


## Writing output of the analytical models back to snowflake ##

In [73]:
# need to reset the index to get rid of error message
sorted_cc_users.index = pd.RangeIndex(start=0, stop=len(sorted_cc_users), step=1)
# convert the pandas df back to snowflake to upload snowflake data warehouse
sf_df = session.create_dataframe(sorted_cc_users)

In [74]:
target_table = 'sorted_cc_users'
sf_df.write.mode("overwrite").save_as_table(target_table)